# LandTrendr in Wicomico County, Maryland
## Author: Sarah McDonald
This codebase produces annual Landsat 8 images from 2013 to 2020, clipped to Wicomico County. The annual data represents the greenest pixel (highest NDVI) during the summer months of each year. LandTrendr is used to identify change information, including magnitude, duration, start year, and end year. LandTrendr uses a single spectral band or index to identify change. This code iterates over each of the native 7 Landsat 8 bands, as well as a few spectral indices, including NDVI and three tasseled cap indices (brightness, greeness, and wetness). For each band or index, an image with 7 bands of change information is exported to Google Drive.

TODO: accuracy assessment code. Extract pixel-based change information for each band/index from AA pixel data.

In [ ]:
# imports
import ee
import geemap

# Initialize the library.
ee.Initialize()

### Define Variables
Define variables used in the define functions

In [ ]:
# prep aoi (wicomico county)
counties =  ee.FeatureCollection("users/au51295/CBW_counties")
wico_bndry = counties.filter(ee.Filter.eq('NAME', 'Wicomico'))

# info for exports
region = ''
crs = ''
transform = ''

# build annual collection
startYear = 2013
endYear = 2020
startDay = '06-15'
endDay = '09-15'

# list of bands of interest - run all through LandTrendr
bands =  ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'NDVI', 'NBR', 'NDMI', 'NDBI', 'TCB', 'TCG', 'TCW']

### Functions
Helper functions to mask clouds, export images to google drive, clip data to Wicomico, add spectral indices as bands to an image, add the image date band, construct an annualized collection of the band of interest, execute LandTrendr, and produce the change images.

#### Functions to add bands and spectral indices to an image.

In [ ]:
def NDVI_helper(img):
    # normalized difference vegetation index
    return img.addBands(img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))

def NBR_helper(img):
    # normalized burn ratio
    return img.addBands(img.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR'))

def NDMI_helper(img):
    # normalized difference moisture index
    return img.addBands(img.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDMI'))

def NDBI_helper(img):
    # normalized difference built up index
    return img.addBands(img.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI'))

def tasseledcap_helper(orig_img):
    img = orig_img.select("SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7")

    # Coefficients are only for Landsat 8 TOA imagery - brightness, greenness, wetness
    coefficients = ee.Array([
        [0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872],
        [-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608],
        [0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559],
    ])

    # create image array
    # Make an Array Image, with a 1-D Array per pixel.
    arrayImage1D = img.toArray()

    # Make an Array Image with a 2-D Array per pixel, 6x1.
    arrayImage2D = arrayImage1D.toArray(1)

    # calculate new bands
    tassled_img = (
        ee.Image(coefficients)
        .matrixMultiply(arrayImage2D)
        .arrayProject([0])
        .arrayFlatten([['TCB', 'TCG', 'TCW']])
    )
        
    # add tasseled cap bands to original image
    orig_img = (
        orig_img
        .addBands(tassled_img.select(['TCB']))
        .addBands(tassled_img.select(['TCG']))
        .addBands(tassled_img.select(['TCW']))
    )
    
    return orig_img

def system_start(img):
  return img.addBands(img.metadata('system:time_start'))


#### Helper functions
Mask clouds, clip images, and export images.

In [ ]:
def maskClouds(img):
  # bit shift operations - QA_PIXEL codes are bitwise values
  cloudDilBitMask = 1 << 1 
  cirrusBitMask = 1 << 2
  cloudsBitMask = 1 << 3
  cloudShadowBitMask = 1 << 4

  # select the QA_PIXEL image
  qa = img.select('QA_PIXEL')

  # Create cloud mask
  mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
            .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
            .And(qa.bitwiseAnd(cloudDilBitMask).eq(0))
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
        )
  
  # mask the clouds from the image
  return img.updateMask(mask)

def export_image(img, name, region, crs, transform):
    # export to google drive
    # Set the export "scale" and "crs" parameters.
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=name,
        folder='LandTrendr_Wicomico',
        region=region,
        crs=crs,
        crsTransform=transform,
        maxPixels=1e10,
        fileFormat="GeoTIFF"
    )

    task.start()

def clip_helper(img):
    return img.clip(wico_bndry)


#### LandTrendr functions
Functions to build the annual collections for LandTrendr, to execute LandTrendr, and to unpack LandTrendr results.

In [ ]:
def build_annual_collection(band_name, wico_bndry, firstPass=False):
    for year in range(startYear, endYear+1):
        img = (
            ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            .filterBounds(wico_bndry)
            .filterDate(f"{year}-{startDay}", f"{year}-{endDay}")
        )
        
        # project county boundary
        if year == startYear and firstPass:
            # print(wico_bndry.geometry().projection().getInfo()['crs'])
            wico_bndry = wico_bndry.geometry().transform(img.first().projection().getInfo()['crs'], 0.001)
            # print(wico_bndry.projection().getInfo()['crs'])
        
        # clip to county
        img = img.map(clip_helper)
        
        # info for exports
        region = img.first().geometry().getInfo()['coordinates']
        crs = img.first().projection().getInfo()['crs']
        transform = img.first().projection().getInfo()['transform']

        # find min system start
        img = img.map(system_start)
    
        # mask clouds
        img = img.map(maskClouds)
        
        # add spectral indices
        img = img.map(NDVI_helper)          # normalized difference vegetation index
        img = img.map(NBR_helper)           # normalized burn ratio
        img = img.map(NDBI_helper)          # normalized difference built up index
        img = img.map(NDMI_helper)          # normalized difference moisture index
        img = img.map(tasseledcap_helper)   # tasseled cap wetness, brightness, and greeness

        # get median start time for the year
        systemStart = img.reduceColumns(ee.Reducer.median(), ['system:time_start']).get('median')
        
        # reduce to greenest image
        img = img.qualityMosaic(band_name).select(band_name)
        img = img.set({
            'year':year,
            'image_list':img,
            'n_bands':img.bandNames().size(),
            'system:time_start': systemStart
        })
        
        # clip mosaic to county -- resets after mosaic
        img = clip_helper(img)
    
        # TODO move this section out to be iterative - no need to build the collection each time
        # convert to image collection for now
        tempCollection = ee.ImageCollection(img.select([band_name]))         

        if year == startYear:
            srCollection = tempCollection
        else:
            srCollection = srCollection.merge(tempCollection)

    # return collection
    return srCollection, region, crs, transform, wico_bndry

def run_landtrendr(annual_collection):
    # run parameters
    run_params = { 
        'timeSeries': annual_collection,
        'maxSegments':            6,
        'spikeThreshold':         0.9,
        'vertexCountOvershoot':   3,
        'preventOneYearRecovery': True,
        'recoveryThreshold':      0.25,
        'pvalThreshold':          0.1,
        'bestModelProportion':    0.75,
        'minObservationsNeeded':  6
        }

    # run LandTrendr
    LTresult = ee.Algorithms.TemporalSegmentation.LandTrendr(
        run_params['timeSeries'],
        run_params['maxSegments'], 
        run_params['spikeThreshold'], 
        run_params['vertexCountOvershoot'], 
        run_params['preventOneYearRecovery'], 
        run_params['recoveryThreshold'], 
        run_params['pvalThreshold'], 
        run_params['bestModelProportion'], 
        run_params['minObservationsNeeded']
    )

    # return results
    return LTresult

def lt_change(LTresult, wico_bndry):
    """
    lt_change Extract the magnitude, duration, start year, end year, start value, end value, and rate of change
              for the largest magnitude change observed.

    Parameters
    ----------
    LTresult : collection
        Results from LandTrendr
    wico_bndry : geometry
        geomery to clip results to

    Returns
    -------
    image
        7-band image of LT results.
    """
    vertexMask = LTresult.select('LandTrendr').arraySlice(0, 3, 4) # slice out the 'Is Vertex' row - yes(1)/no(0)
    vertices = LTresult.select('LandTrendr').arrayMask(vertexMask) # use the 'Is Vertex' row as a mask for all rows
    left = vertices.arraySlice(1, 0, -1)    # slice out the vertices as the start of segments
    right = vertices.arraySlice(1, 1, None) # slice out the vertices as the end of segments
    startYear = left.arraySlice(0, 0, 1)    # get year dimension of LT data from the segment start vertices
    startVal = left.arraySlice(0, 2, 3)     # get spectral index dimension of LT data from the segment start vertices
    endYear = right.arraySlice(0, 0, 1)     # get year dimension of LT data from the segment end vertices 
    endVal = right.arraySlice(0, 2, 3)      # get spectral index dimension of LT data from the segment end vertices
    dur = endYear.subtract(startYear)       # subtract the segment start year from the segment end year to calculate the duration of segments 
    mag = endVal.subtract(startVal)         # substract the segment start index value from the segment end index value to calculate the delta of segments
    rate = mag.divide(dur)                  # calculate the rate of spectral change

    segInfo = (
        ee.Image.cat([startYear.add(1), endYear, startVal, endVal, mag, dur, rate])
        .toArray(0)
        .mask(vertexMask.mask())
    )

    # greatest vegetation loss
    chg_sort = segInfo.arraySlice(0,4,5).toArray(0).multiply(-1) # need to flip the delta here, since arraySort is working by ascending order
    seg_chg_sorted = segInfo.arraySort(chg_sort) # sort the array by magnitude
    chg_greatest = seg_chg_sorted.arraySlice(1, 0, 1) # get the first segment in the sorted array (greatest magnitude vegetation loss segment)

    distDir = 1 # scalar to invert values if needed

    # construct single image
    chgImg = ee.Image.cat(chg_greatest.arraySlice(0,0,1).arrayProject([1]).arrayFlatten([['yod']]),
                                chg_greatest.arraySlice(0,1,2).arrayProject([1]).arrayFlatten([['endYr']]),
                                chg_greatest.arraySlice(0,2,3).arrayProject([1]).arrayFlatten([['startVal']]).multiply(distDir),
                                chg_greatest.arraySlice(0,3,4).arrayProject([1]).arrayFlatten([['endVal']]).multiply(distDir),
                                chg_greatest.arraySlice(0,4,5).arrayProject([1]).arrayFlatten([['mag']]).multiply(distDir),
                                chg_greatest.arraySlice(0,5,6).arrayProject([1]).arrayFlatten([['dur']]),
                                chg_greatest.arraySlice(0,6,7).arrayProject([1]).arrayFlatten([['rate']]).multiply(distDir))

    return chgImg.clip(wico_bndry)


### Exectute LandTrendr
Iterate over all bands and indices of interest and:
1. Build annual collection of images with the band of interest
2. Execute LandTrendr.
3. Produce change image.
4. Export the change image.

In [ ]:
# iterate over the bands of interest
for b in bands:
    print(b)

    # build annual collection for the index of interest
    print("\tBuilding Collection")
    if bands.index(b) == 0:
        srCollection, region, crs, transform, wico_bndry = build_annual_collection(b, wico_bndry, firstPass=True)
    else:
        srCollection, region, crs, transform, wico_bndry = build_annual_collection(b, wico_bndry)

    # run LandTrendr for the band
    print("\tRunning LandTrendr")
    lt = run_landtrendr(srCollection)

    # produce image of change information (magnitude, duration, start year, etc.)
    print("\tCreating Change")
    # wico_bndry = wico_bndry.geometry().transform(srCollection.first().projection().getInfo()['crs'], 0.001)
    chg_img = lt_change(lt, wico_bndry)

    # export the image
    print("\tExporting Change Results")
    export_image(chg_img, f"LT_Wico_{b}", region, crs, transform)

    